## Parcels 3D

In [24]:
from parcels import FieldSet, ParticleSet, JITParticle, AdvectionRK4_3D, ErrorCode, ParticleFile, Variable, convert_IndexedOutputToArray
from datetime import timedelta as delta
from progressbar import ProgressBar
import numpy as np
from os import path
import math
from datetime import timedelta
from datetime import datetime

ufiles = ["/Users/hart-davis/Downloads/OceanParcels-Parcelsv0.9Paper_Scripts/uvel03166.nc"]
vfiles = ["/Users/hart-davis/Downloads/OceanParcels-Parcelsv0.9Paper_Scripts/uvel03166.nc"]
wfiles = ["/Users/hart-davis/Downloads/OceanParcels-Parcelsv0.9Paper_Scripts/wvel03165.nc"]
tfiles = ["/Users/hart-davis/Downloads/OceanParcels-Parcelsv0.9Paper_Scripts/temp03165.nc"]


variables = {'U': 'uvel', 'V': 'uvel', 'W': 'wvel', 'temp': 'temp'}
dimensions = {'lat': 'lat', 'lon': 'lon', 'time': 'time', 'depth': 'lev'}
    
    
filenames = {'U': ufiles, 'V': vfiles, 'W': wfiles, 'temp':tfiles}
fieldset = FieldSet.from_netcdf(filenames, variables, dimensions)

pset = ParticleSet.from_line(fieldset=fieldset, size=5, pclass=JITParticle, 
                             start=(14.8504999999999999, -32.9804000000000002), finish=(14.8504999999999999, -32.9804000000000002))

def set_ofes_fieldset(snapshots):
    ufiles = [path.join(path.dirname(__file__), "ofesdata", "uvel{:05d}.nc".format(s)) for s in snapshots]
    vfiles = [path.join(path.dirname(__file__), "ofesdata", "vvel{:05d}.nc".format(s)) for s in snapshots]
    wfiles = [path.join(path.dirname(__file__), "ofesdata", "wvel{:05d}.nc".format(s)) for s in snapshots]
    tfiles = [path.join(path.dirname(__file__), "ofesdata", "temp{:05d}.nc".format(s)) for s in snapshots]
    filenames = {'U': ufiles, 'V': vfiles, 'W': wfiles, 'temp': tfiles}
    variables = {'U': 'uvel', 'V': 'vvel', 'W': 'wvel', 'temp': 'temp'}
    dimensions = {'lat': 'lat', 'lon': 'lon', 'time': 'time', 'depth': 'lev'}

    fieldset = FieldSet.from_netcdf(filenames, variables, dimensions)
    fieldset.U.data /= 100.  # convert from cm/s to m/s
    fieldset.V.data /= 100.  # convert from cm/s to m/s
    fieldset.W.data /= 100.  # convert from cm/s to m/s
    return fieldset

output_file = pset.ParticleFile(name="testing3")
pset.execute(AdvectionRK4_3D, starttime=datetime(2008, 1, 1), runtime=timedelta(days=70),
             dt=timedelta(minutes=5), interval=timedelta(hours=6),output_file=output_file)

plotTrajectoriesFile('testing3.nc')

def SampleTemp(particle, fieldset, time, dt):
    particle.temp = fieldset.temp[time, particle.lon, particle.lat, particle.depth]


def Sink(particle, fieldset, time, dt):
    if particle.depth > fieldset.dwellingdepth:
        particle.depth = particle.depth + fieldset.sinkspeed * dt
    else:
        particle.depth = fieldset.dwellingdepth


def Age(particle, fieldset, time, dt):
    if particle.depth <= fieldset.dwellingdepth:
        particle.age = particle.age + math.fabs(dt)
    if particle.age > fieldset.maxage:
        particle.delete()


def DeleteParticle(particle, fieldset, time, dt):
    particle.delete()


def run_corefootprintparticles(outfile):
    snapshots = range(3165, 3289)
    fieldset = set_ofes_fieldset(snapshots[-4:-1])
    fieldset.add_constant('dwellingdepth', 50.)
    fieldset.add_constant('sinkspeed', 200./86400)
    fieldset.add_constant('maxage', 30.*86400)

    corelon = [17.30]
    corelat = [-34.70]
    coredepth = [2440]

    class ForamParticle(JITParticle):
        temp = Variable('temp', dtype=np.float32, initial=np.nan)
        age = Variable('age', dtype=np.float32, initial=0.)

    pset = ParticleSet(fieldset=fieldset, pclass=ForamParticle, lon=corelon, lat=corelat,
                       depth=coredepth, time=fieldset.U.time[-1])
    pfile = ParticleFile(outfile, pset, type="indexed")
    pfile.write(pset, pset[0].time)

    kernels = pset.Kernel(AdvectionRK4_3D) + Sink + SampleTemp + Age

    pbar = ProgressBar()
    for s in pbar(range(len(snapshots)-5, -1, -1)):
        pset.execute(kernels, starttime=pset[0].time, runtime=delta(days=3),
                     dt=delta(minutes=-5), interval=delta(days=-1),
                     recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle})

        pset.add(ForamParticle(lon=corelon, lat=corelat, depth=coredepth, fieldset=fieldset))
        pfile.write(pset, pset[0].time)
        fieldset.advancetime(set_ofes_fieldset([snapshots[s]]))


def make_plot(trajfile):
    from netCDF4 import Dataset
    import matplotlib.pyplot as plt
    from mpl_toolkits.basemap import Basemap

    class ParticleData(object):
        def __init__(self):
            self.id = []

    def load_particles_file(fname, varnames):
        T = ParticleData()
        pfile = Dataset(fname, 'r')
        T.id = pfile.variables['trajectory'][:]
        for v in varnames:
            setattr(T, v, pfile.variables[v][:])
        return T

    T = load_particles_file(trajfile, ['lon', 'lat', 'temp', 'z'])
    m = Basemap(projection='merc', llcrnrlat=-40, urcrnrlat=-27.5, llcrnrlon=10, urcrnrlon=32.5, resolution='h')
    m.drawcoastlines()
    m.fillcontinents(color='burlywood')
    m.drawparallels(np.arange(-50, -20, 10), labels=[True, False, False, False])
    m.drawmeridians(np.arange(0, 40, 10), labels=[False, False, False, True])

    sinks = np.where(T.z > 50.)
    dwell = np.where(T.z == 50.)
    xs, ys = m(T.lon[dwell], T.lat[dwell])
    m.scatter(xs, ys, c=T.temp[dwell], s=5)
    cbar = plt.colorbar()
    cbar.ax.xaxis.set_label_position('top')
    cbar.ax.set_xlabel('[$^\circ$C]')

    xs, ys = m(T.lon[sinks], T.lat[sinks])
    m.scatter(xs, ys, c='k', s=5)
    xs, ys = m(T.lon[0, 0], T.lat[0, 0])
    m.plot(xs, ys, 'om')
    plt.show()


outfile = "corefootprint_particles"
run_corefootprintparticles(outfile)
convert_IndexedOutputToArray(file_in=outfile+".nc", file_out=outfile+"_array.nc")
make_plot(outfile+"_array.nc")


INFO: Note that positive vertical velocity is assumed DOWNWARD by AdvectionRK4_3D
INFO: Compiled JITParticleAdvectionRK4_3D ==> /var/folders/r4/54mc4t2559d_d1fnkl99m38r0000gn/T/parcels-501/58a96d93bd99b72479d9225b9a7db11a.so
Exception AttributeError: "'ParticleFile' object has no attribute 'dataset'" in <bound method ParticleFile.__del__ of <parcels.particlefile.ParticleFile object at 0x11eebe5d0>> ignored


OutOfTimeError: 0
Particle P[350](lon=14.850500, lat=-32.980400, depth=2.500000, time=220838400.000000)[xi=98, yi=120, zi=0]
Time: 2008-01-01 00:00:00,	timestep dt: 300.000000
Field sampled outside time domain at time 2008-01-01 00:00:00. Try setting allow_time_extrapolation to True

In [28]:
fieldset.W.data

array([[[[ -1.35241702e-04,   2.52118014e-04,   5.61634544e-04, ...,
            4.99505957e-04,   4.40952455e-04,   2.29972895e-04],
         [ -9.97841780e-05,   2.51544261e-04,   5.90122887e-04, ...,
            4.04263003e-04,   4.58469905e-04,   3.71983653e-04],
         [ -1.51261047e-04,   1.62213939e-04,   5.04133932e-04, ...,
            3.19576997e-04,   3.36788071e-04,   3.55817057e-04],
         ..., 
         [  1.01346915e-04,   1.15231314e-05,  -7.87469980e-05, ...,
            8.66716960e-04,  -1.07412554e-04,  -1.69298879e-03],
         [  5.94158992e-05,  -4.29088323e-05,  -1.22793732e-04, ...,
            8.57141335e-04,   4.02469886e-04,  -9.89356777e-04],
         [  2.86050249e-06,  -9.39593228e-05,  -1.73311302e-04, ...,
           -3.20276711e-04,   4.83675190e-04,   2.11895909e-04]],

        [[ -2.82350258e-04,   5.14778483e-04,   1.16166845e-03, ...,
            1.04510284e-03,   9.21465864e-04,   4.78578324e-04],
         [ -2.07698104e-04,   5.18689805e-04,